In [1]:
import pandas as pd
import numpy as np
import psycopg2
import scipy

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.stats as stats
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression

In [2]:
parcel_df = pd.read_csv('../../data/EXTR_Parcel.csv', encoding='latin-1')
res_bldg_df = pd.read_csv('../../data/EXTR_ResBldg.csv', encoding='latin-1')
rp_sale_df = pd.read_csv('../../data/EXTR_RPSale.csv', encoding='latin-1')

/Users/sam/opt/anaconda3/envs/linreg-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/sam/opt/anaconda3/envs/linreg-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Code to prepend zeros for major and minor PINs
dfs = [rp_sale_df, res_bldg_df, parcel_df]
for df in dfs:
    df['Major'] = df['Major'].astype('str').str.zfill(6)
    df['Minor'] = df['Minor'].astype('str').str.zfill(4)
for df in dfs:
    df['Major_+_Minor'] = df['Major'] + df['Minor']

In [19]:
combined_df = pd.merge(parcel_df, res_bldg_df, on='Major_+_Minor')
combined_df = pd.merge(combined_df, rp_sale_df, on='Major_+_Minor')

In [15]:
"""
selecting interested columns
"""
groomed_df = combined_df[['Major_+_Minor', 'SalePrice', 'SqFtTotLiving', 'SqFtOpenPorch', 
                          'SqFtEnclosedPorch', 'SqFtDeck', 'TidelandShoreland',
                          'TrafficNoise', 'AirportNoise', 'PowerLines', 'Stories', 'Condition', 'Area', 
                          'DocumentDate', 'TrafficNoise', 'AirportNoise', 'PowerLines', 'OtherNuisances']]

In [18]:
"""
filtering data
"""
groomed_df = groomed_df[groomed_df['SalePrice'] > 0]
groomed_df = groomed_df[groomed_df['SalePrice'] < 5000000]
#groomed_df = groomed_df[groomed_df['LivableSpace'] < 50000]
groomed_df = groomed_df[groomed_df['SqFtTotLiving'] < 50000]
groomed_df = groomed_df[groomed_df['DocumentDate'].str.contains('2019')]
# groomed_df['TrafficNoise'] = groomed_df['TrafficNoise'].astype(bool)

KeyError: 'SalePrice'

In [17]:
fsm_multi = ols(formula='SalePrice ~ SqFtTotLiving + SqFtOpenPorch + SqFtDeck + TidelandShoreland + Condition + Stories + Area + TrafficNoise', data=groomed_df).fit()
fsm_multi.summary()

PatsyError: Error evaluating factor: NameError: name 'SalePrice' is not defined
    SalePrice ~ SqFtTotLiving + SqFtOpenPorch + SqFtDeck + TidelandShoreland + Condition + Stories + Area + TrafficNoise
    ^^^^^^^^^